<font color='red'><h1> Workbook to deploy the best strategy </h1></font>

This workbook deploys the best strategy, which includes the top 20 features ranked by BorutaPy library and XGBoost regressor with hyperparameter tuning.

## Import libraries

In [1]:
import numpy as np
import pandas as pd
import os
import xgboost as xgb
from sklearn.preprocessing import StandardScaler

In [2]:
csv_path = os.getcwd() + "/data/reserved/"

In [3]:
train_df = pd.read_csv(csv_path + "train.csv")
test_df = pd.read_csv(csv_path + "test.csv")
feature_df = pd.read_csv(os.getcwd() + '/data/ranked_features.csv')

## Preprocessing

In [4]:
cols = feature_df[feature_df['Rank'] <= 10]['Feature'].values

In [5]:
test_df = test_df[cols]
train_df = train_df[np.append(cols,['emission'])]

In [6]:
for col in test_df.columns:
    train_df[col] = train_df[col].fillna(train_df[col].median())
    test_df[col] = test_df[col].fillna(test_df[col].median())

In [7]:
X_train = train_df.drop('emission',axis=1)
y_train = train_df['emission']

In [8]:
scaler = StandardScaler()

In [9]:
X_train = scaler.fit_transform(X_train)
test_df = scaler.fit_transform(test_df)

In [10]:
X_train = pd.DataFrame(X_train, columns=train_df.drop('emission',axis=1).columns)
test_df = pd.DataFrame(test_df, columns=train_df.drop('emission',axis=1).columns)

## Deploy the model

In [17]:
xgb_model = xgb.XGBRegressor(objective='reg:squarederror',n_estimators=100,learning_rate=0.1,max_depth=13,random_state=42)
xgb_model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=13, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [19]:
y_pred = xgb_model.predict(test_df)

## Make submission

In [33]:
df = pd.read_csv(csv_path + 'test.csv')
df = df[df.columns[0:2]]

In [34]:
df['emission'] = y_pred
df.drop('latitude',axis=1,inplace=True)

In [40]:
df.to_csv(csv_path + "submission.csv",index=False)